In [20]:
import numpy as np
import pandas as pd
from tqdm import tqdm

import json
import re
import string
import pymorphy2
import random

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer  
stop_words = stopwords.words('english')

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,classification_report
from sklearn import neighbors

In [2]:
morph = pymorphy2.MorphAnalyzer()

In [3]:
with open('Data/ranking_train.jsonl', 'r') as f:
    target = [json.loads(line) for line in f]

rows = []
for d in target:
    text = d['text']
    comments = [c['text'] for c in d['comments']]
    score = [c['score'] for c in d['comments']]
    rows.append({'text': text, 'comments': comments, 'scores':score})

df = pd.DataFrame(rows)

In [4]:
df = df.explode('comments')

In [5]:
unknown = []
for d in target:
    for c in d['comments']:
        unknown.append(c['score'])
df['scores'] = unknown

In [6]:
df.reset_index(drop=True, inplace = True)

In [7]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df = df.sort_values('text').reset_index(drop=True)

In [9]:
df['comments'] = df['comments'].replace("[0-9!#()$\,\'\-\.*+/:;<=>?@[\]^_`{|}\"]+", ' ', regex=True)
df['comments'] = df['comments'].replace(r'\s+', ' ', regex=True)
df['comments'] = df['comments'].apply(lambda x: ' '.join([word for word in x.split() if word not in stopwords.words('english')]))
df['comments'] = df['comments'].apply(lambda x: ' '.join([morph.parse(word)[0]. normal_form for word in x.split()]))

Поделим выборку на тренировочную и тестовую, перемешивать не будем, 
так как комментарии уже перемешаны в отношении текстов

In [46]:
X_train, X_test, y_train, y_test = train_test_split(
    df['comments'].values,df['scores'].values,test_size=0.2,shuffle=False)

In [47]:
tfidf_vectorizer = TfidfVectorizer() 

tfidf_train_vectors = tfidf_vectorizer.fit_transform(X_train)

tfidf_test_vectors = tfidf_vectorizer.transform(X_test)

KNeighborsClassifier

In [57]:
classifier_knn = neighbors.KNeighborsClassifier()

classifier_knn.fit(tfidf_train_vectors,y_train)

KNeighborsClassifier()

In [58]:
y_pred_knn = classifier_knn.predict(tfidf_test_vectors)

/Users/speedyracer/opt/anaconda3/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:237: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [67]:
y_pred_knn_ndcg = y_pred_knn[7:].reshape((-1,5))

In [68]:
Y_test_ndcg = y_test[7:].reshape((-1,5))

In [59]:
print(classification_report(y_test,y_pred_knn))

              precision    recall  f1-score   support

           0       0.20      0.03      0.05     17621
           1       0.20      0.04      0.07     17621
           2       0.21      0.06      0.09     17622
           3       0.20      0.23      0.21     17621
           4       0.20      0.66      0.31     17622

    accuracy                           0.20     88107
   macro avg       0.20      0.20      0.15     88107
weighted avg       0.20      0.20      0.15     88107



In [69]:
from sklearn.metrics import ndcg_score
ndcg_score(Y_test_ndcg, y_pred_knn_ndcg)

0.8056971638300708

LogisticRegression

In [48]:
classifier_log = LogisticRegression(max_iter=10000)

classifier_log.fit(tfidf_train_vectors,y_train)

LogisticRegression(max_iter=10000)

In [49]:
y_pred_log = classifier_log.predict(tfidf_test_vectors)

In [62]:
y_pred_log_ndcg = y_pred_log[7:].reshape((-1,5))

In [64]:
Y_test_ndcg = y_test[7:].reshape((-1,5))

In [50]:
print(classification_report(y_test,y_pred_log))

              precision    recall  f1-score   support

           0       0.32      0.46      0.38     17621
           1       0.22      0.18      0.20     17621
           2       0.22      0.17      0.19     17622
           3       0.23      0.20      0.22     17621
           4       0.28      0.30      0.29     17622

    accuracy                           0.26     88107
   macro avg       0.25      0.26      0.26     88107
weighted avg       0.25      0.26      0.26     88107



In [65]:
from sklearn.metrics import ndcg_score
ndcg_score(Y_test_ndcg, y_pred_log_ndcg)

0.8541465937957847

RandomForestClassifier

In [51]:
classifier_for = RandomForestClassifier()

classifier_for.fit(tfidf_train_vectors,y_train)

RandomForestClassifier()

In [52]:
y_pred_for = classifier_for.predict(tfidf_test_vectors)

In [70]:
y_pred_for_ndcg = y_pred_for[7:].reshape((-1,5))

In [71]:
Y_test_ndcg = y_test[7:].reshape((-1,5))

In [53]:
print(classification_report(y_test,y_pred_for))

              precision    recall  f1-score   support

           0       0.33      0.49      0.40     17621
           1       0.22      0.14      0.17     17621
           2       0.21      0.14      0.17     17622
           3       0.23      0.19      0.21     17621
           4       0.28      0.39      0.33     17622

    accuracy                           0.27     88107
   macro avg       0.25      0.27      0.25     88107
weighted avg       0.25      0.27      0.25     88107



In [72]:
from sklearn.metrics import ndcg_score
ndcg_score(Y_test_ndcg, y_pred_for_ndcg)

0.8583971568964195